#Importing modules

In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#Downloading and Slicing the Dataset

In [2]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",
                                                   split=('train[:60%]','train[60%:]', 'test'),
                                                   as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUV023S/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUV023S/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


#Downloading the Embeddings layer

In [4]:
hub_layer = hub.KerasLayer('https://tfhub.dev/google/nnlm-en-dim50/2', input_shape=[], 
                           dtype=tf.string, trainable=True)

#Creating the model

In [5]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

#Training the model

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 21s 642ms/step - loss: 0.6811 - accuracy: 0.5021 - val_loss: 0.6405 - val_accuracy: 0.5524
Epoch 2/10
30/30 [==============================] - 20s 639ms/step - loss: 0.5835 - accuracy: 0.6597 - val_loss: 0.5415 - val_accuracy: 0.6903
Epoch 3/10
30/30 [==============================] - 20s 641ms/step - loss: 0.4577 - accuracy: 0.7839 - val_loss: 0.4367 - val_accuracy: 0.8020
Epoch 4/10
30/30 [==============================] - 20s 643ms/step - loss: 0.3300 - accuracy: 0.8636 - val_loss: 0.3563 - val_accuracy: 0.8407
Epoch 5/10
30/30 [==============================] - 20s 644ms/step - loss: 0.2307 - accuracy: 0.9110 - val_loss: 0.3216 - val_accuracy: 0.8611
Epoch 6/10
30/30 [==============================] - 19s 634ms/step - loss: 0.1672 - accuracy: 0.9436 - val_loss: 0.3078 - val_accuracy: 0.8674
Epoch 7/10
30/30 [==============================] - 20s 632ms/step - loss: 0.1221 - accuracy: 0.9629 - val_loss: 0.3050 - val_accuracy: 0.8710

In [9]:
model.evaluate(test_data.batch(512), verbose=2)

49/49 - 5s - loss: 0.3618 - accuracy: 0.8524


[0.3618004024028778, 0.8523600101470947]